In [2]:
import os
import json
from openai import OpenAI
from IPython.display import display, JSON

OPENAI_API_KEY_PATH = '/home/loc/Documents/OPENAI_API_KEY.txt'
llm_model = 'gpt-3.5-turbo-0613'

with open(OPENAI_API_KEY_PATH) as f:
    OPENAI_API_KEY = f.read().strip()
    os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

client = OpenAI(api_key = OPENAI_API_KEY)

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

# Simple Chain

In [4]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)

model = ChatOpenAI()

output_parser = StrOutputParser()

/home/loc/miniconda3/envs/venv/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [5]:
chain = prompt | model | output_parser

In [6]:
chain.invoke({"topic":"bears"})

'Why did the bear bring a flashlight to the party? \n\nBecause he wanted to be the light of the bear-y!'

## More complex chain

And Runnable Map to supply user-provided inputs to the prompt.

In [7]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [8]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

/home/loc/miniconda3/envs/venv/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/home/loc/miniconda3/envs/venv/lib/python3.8/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [9]:
retriever.get_relevant_documents("where did harrison work?")

/home/loc/miniconda3/envs/venv/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

In [10]:
retriever.get_relevant_documents("what do bears like to eat")

[Document(page_content='bears like to eat honey'),
 Document(page_content='harrison worked at kensho')]

In [11]:
template = """
Answer the question based only on the following context:
{content}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

prompt

ChatPromptTemplate(input_variables=['content', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['content', 'question'], template='\nAnswer the question based only on the following context:\n{content}\n\nQuestion: {question}\n'))])

In [22]:
from langchain.schema.runnable import RunnableMap

In [23]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [24]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [25]:
inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(page_content='harrison worked at kensho'),
  Document(page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

In [28]:
chain = inputs | prompt | model | output_parser

In [29]:
chain.invoke({"question": "where did harrison work?"})

'Harrison worked at Kensho.'

# Bind

In [22]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [23]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human","{input}")
    ]
)

model = ChatOpenAI(temperature=0).bind(functions=functions)

In [24]:
runnable = prompt | model

In [25]:
runnable.invoke({"input": "what is the weather in sf"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'weather_search'}}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 64, 'total_tokens': 80}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-6ee07ec7-6d91-4422-8d9e-521d3e498a0d-0')

In [26]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [28]:
model = model.bind(functions=functions)

In [29]:
runnable = prompt | model

In [30]:
runnable.invoke({"input": "how did the patriots do yesterday?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"team_name":"New England Patriots"}', 'name': 'sports_search'}}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 99, 'total_tokens': 117}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-870d7f29-cffe-4295-9d7a-9bdd72fbecd8-0')

# Fallbacks

In [31]:
import json
from langchain.llms import OpenAI

In [32]:
simple_model = OpenAI(
    temperature=0, 
    max_tokens=1000, 
    model="gpt-3.5-turbo-instruct"
)
simple_chain = simple_model | json.loads

/home/loc/miniconda3/envs/venv/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [33]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [36]:
simple_model.invoke(challenge)

'\n\n{\n    "title": "Autumn Leaves",\n    "author": "Emily Dickinson",\n    "first_line": "The leaves are falling, one by one"\n}\n\n{\n    "title": "The Ocean\'s Song",\n    "author": "Pablo Neruda",\n    "first_line": "I hear the ocean\'s song, a symphony of waves"\n}\n\n{\n    "title": "A Winter\'s Night",\n    "author": "Robert Frost",\n    "first_line": "The snow falls softly, covering the ground"\n}'

In [38]:
model = ChatOpenAI(temperature=0)
chain = model | StrOutputParser() | json.loads

In [39]:
chain.invoke(challenge)

{'poem1': {'title': 'The Rose',
  'author': 'Emily Dickinson',
  'firstLine': 'A rose is a rose is a rose'},
 'poem2': {'title': 'The Road Not Taken',
  'author': 'Robert Frost',
  'firstLine': 'Two roads diverged in a yellow wood'},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'firstLine': 'Hope is the thing with feathers'}}

In [40]:
final_chain = simple_chain.with_fallbacks([chain])

In [41]:
final_chain.invoke(challenge)

{'poem1': {'title': 'The Rose',
  'author': 'Emily Dickinson',
  'firstLine': 'A rose is a rose is a rose'},
 'poem2': {'title': 'The Road Not Taken',
  'author': 'Robert Frost',
  'firstLine': 'Two roads diverged in a yellow wood'},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'firstLine': 'Hope is the thing with feathers'}}

# Interface

In [42]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)

model = ChatOpenAI()

output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [43]:
chain.invoke({"topic": "bears"})

"Why don't bears like fast food? Because they can't catch it!"

In [44]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

['Why do bears have hairy coats?\n\nFur protection!',
 'Why are frogs so happy? Because they eat whatever bugs them!']

In [45]:
for t in chain.stream({'topic':'bears'}):
    print(t)


Why
 don
't
 bears
 like
 fast
 food
?
 Because
 they
 can
't
 catch
 it
!



In [46]:
response = await chain.ainvoke({"topic": "bears"})
response

'Why did the bear bring toilet paper to the party? \n\nBecause he heard it was a bear necessity!'